# Inversão da velocidade de grupo de ondas Love

## Carregando os dados

In [ ]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
periodo, velocidade = np.loadtxt('vel-grupo-sao-vicente.txt', unpack=True)
velocidade *= 1000

In [ ]:
plt.plot(periodo, velocidade, 'ok')

## Modelagem direta

In [ ]:
def vel_grupo_love(velocidade, h, v1, v2, den1, den2):
    """
    Calcula os períodos dada as velocidades de grupo e um modelo.
    
    O modelo eh de uma camada sobre um meio semi-infinito.
    """
    cima = 2*np.pi*h*np.sqrt(velocidade**2/v1**2 - 1)/velocidade
    baixo = np.arctan2(den2*np.sqrt(v2**4 - velocidade**2*v2**2),
                       den1*np.sqrt(velocidade**2*v1**2 - v1**4))
    periodo = cima/baixo
    return periodo

In [ ]:
modelo = vel_grupo_love(velocidade, 40000, 3500, 6000, 2670, 3200)
plt.plot(periodo, velocidade, 'ok')
plt.plot(modelo, velocidade, '.-r')

## Inversão

In [ ]:
from fatiando.inversion import Misfit

In [ ]:
class GrupoLove(Misfit):
    
    def __init__(self, perido, velocidade, den1, den2):
        super(GrupoLove, self).__init__(data=periodo, nparams=3, islinear=False)
        self.velocidade = velocidade
        self.den1 = den1
        self.den2 = den2
    
    def predicted(self, p):
        h, v1, v2 = p
        periodo = vel_grupo_love(self.velocidade, h, v1, v2, self.den1, self.den2)
        return periodo
    
    def jacobian(self, p):
        pass        

### Assume densidades de crosta-manto

In [ ]:
amor = GrupoLove(periodo, velocidade, den1=2700,  den2=3200)

In [ ]:
amor.config('acor', bounds=[1000, 70000, 1000, 7000, 1000, 7000]).fit()

In [ ]:
amor.estimate_

In [ ]:
plt.plot(periodo, velocidade, 'ok')
plt.plot(amor.predicted(), velocidade, '.-r')

## Assume densidades de bacia-embasamento

In [ ]:
amor2 = GrupoLove(periodo, velocidade, den1=2000,  den2=2700)

In [ ]:
amor2.config('acor', bounds=[1000, 10000, 1000, 7000, 1000, 7000]).fit()

In [ ]:
amor2.estimate_

In [ ]:
plt.plot(periodo, velocidade, 'ok')
plt.plot(amor2.predicted(), velocidade, '.-r')